In [3]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema import Document
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline ,HuggingFaceEmbeddings

from langchain.prompts import PromptTemplate
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import (
    FewShotPromptTemplate,
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate
)
import bitsandbytes as bnb
import pickle
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
def get_embedding():
    embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-small',
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
        )
    return embeddings

def load_and_vectorize(csv_path, db_path):
    if os.path.exists(db_path) and os.path.exists(db_path + '_metadata.pkl'):
        print("Loading FAISS DB from:", db_path)
        db, metadata = load_faiss_db(db_path)
        return db

    # CSV 파일을 불러와 데이터프레임 생성
    train_df = pd.read_csv(csv_path)
    train_df.drop('SAMPLE_ID', axis=1, inplace=True)
    trainset = train_df.to_dict(orient='records')
    print("Loaded Fewshot Set:", trainset[:3])
    
    # 벡터화할 텍스트 생성
    to_vectorize = ["\n\n".join(example.values()) for example in trainset]
    
    # 벡터화 및 FAISS DB 생성
    fewshow_vectordb = FAISS.from_texts(to_vectorize, embedding=get_embedding(), metadatas=trainset)
    
    # FAISS DB 저장
    save_faiss_db(fewshow_vectordb, db_path)
    
    return fewshow_vectordb

def save_faiss_db(db, db_path):
    db.save_local(db_path)
    with open(db_path + '_metadata.pkl', 'wb') as f:
        pickle.dump(db, f)

def load_faiss_db(db_path):
    db = FAISS.load_local(db_path, embeddings=get_embedding(), allow_dangerous_deserialization=True)
    with open(db_path + '_metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
    return db, metadata

def load_chunks_make_docdb(pdf_directory, db_path):
    if os.path.exists(db_path) and os.path.exists(db_path + '_metadata.pkl'):
        print("Loading FAISS DB from:", db_path)
        db, metadata = load_faiss_db(db_path)
        return db

    print("Loading PDF files from:", pdf_directory)
    documents = []

    # PDF 파일들을 로드하여 분할
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf').replace('\\', '/'))
    for pdf_file in pdf_files:
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        documents.extend(pdf_documents)
    
    # 분할된 텍스트를 벡터로 변환하여 FAISS DB에 저장
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = chunk_splitter.split_documents(documents)
    print("Done.", len(chunks), "chunks")
    
    print("Creating FAISS DB")
    # FAISS DB 생성 및 저장
    db = FAISS.from_documents(chunks, embedding=get_embedding())
    save_faiss_db(db, db_path)
    print("Done.")
    
    return db

def make_fewshot_prompt(fewshot_vectordb, k = 3):
    # Semantic Similarity Example Selector 설정
    example_prompt = PromptTemplate.from_template("Question: {Question}\nAnswer: {Answer}\nSource: {Source}")

    example_selector = SemanticSimilarityExampleSelector(
        vectorstore=fewshot_vectordb,
        k=k,
    )

    # FewShotPromptTemplate 생성
    fewshot_prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        suffix="Question: {input}",
        input_variables=["input"],
    )
    return fewshot_prompt

def make_dict(dir='train.csv'):
    df = pd.read_csv(dir)
    df.drop('SAMPLE_ID', axis=1, inplace=True)
    
    return df.to_dict(orient='records')

def setup_llm_pipeline(model_id):
    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False
    
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
    )
    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        #trust_remote_code=True 
        )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        #model_kwargs={"torch_dtype": torch.bfloat16},
        do_sample = True,
        temperature=0.1,
        top_p=0.5,
        return_full_text=False,
        # eos_token_id=terminators,
        max_new_tokens=128,
        pad_token_id=tokenizer.eos_token_id  # 패딩 토큰을 EOS 토큰 ID로 설정
    )


    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm

In [4]:
fewshot_db = load_and_vectorize('train.csv', './fewshot_faiss_db')
fewshot_prompt = make_fewshot_prompt(fewshot_db)

train_db = load_chunks_make_docdb('./train_source', './train_faiss_db')
train_retriever = train_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1})

test_db = load_chunks_make_docdb('./test_source', './test_faiss_db')
test_retriver = test_db.as_retriever(search_type = "mmr",search_kwargs={'k': 3})

train_dict = make_dict('train.csv')
test_dict = make_dict('test.csv')

Loading FAISS DB from: ./fewshot_faiss_db
Loading FAISS DB from: ./train_faiss_db
Loading FAISS DB from: ./test_faiss_db


In [3]:
# i=2
# def get_example(fewshot_db,k=3):
#     example_selector = SemanticSimilarityExampleSelector(
#         vectorstore=fewshot_db,
#         k=3,
#     )

#     # The prompt template will load examples by passing the input do the `select_examples` method
#     ex = example_selector.select_examples({"input": test_dict[i]['Question']})

#     return ex
# few_shot_prompt = get_example(fewshot_db,k=3)
# print(few_shot_prompt)
# from langchain_core.messages import AIMessage, HumanMessage, ToolMessage , merge_content
# def concat_context(ex,train_retriever):
#     for i , entry in enumerate(ex):
#         print('Q:', i)
#         context = train_retriever.invoke(entry['Question'])
        
#         ex[i]['Question'] = context[0].page_content + '\n\n' + entry['Question']
#         print(entry['Question'])    
#     return ex

# few_shot_prompt = concat_context(few_shot_prompt,train_retriever)

# final_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are a wondrous wizard of math."),
#         few_shot_prompt,
#         ("human", "{context}\n\n{input}"),
#     ]
# )
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)
# llm = setup_llm_pipeline('yanolja/EEVE-Korean-Instruct-2.8B-v1.0')

# chain = (
#     {
#         "context": test_retriver | format_docs,
#         "input": RunnablePassthrough(),
#     }
#     |final_prompt 
#     | llm
    
#     )

# chain.invoke({"input": test_dict[i]['Qusetion']})

In [5]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
i=2
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
llm = setup_llm_pipeline('meta-llama/Meta-Llama-3.1-8B-Instruct')


Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.89s/it]


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [1]:
def fewshot_rag(llm, fewshot_db,test_retriver,question):
    example_selector = SemanticSimilarityExampleSelector(
        vectorstore=fewshot_db,
        k=3,
    )

    # The prompt template will load examples by passing the input do the `select_examples` method
    # example_selector.select_examples({"input": "horse"})
    # Define the few-shot prompt.
    few_shot_prompt = FewShotChatMessagePromptTemplate(
        # The input variables select the values to pass to the example_selector
        input_variables=["input"],
        example_selector=example_selector,
        # Define how each example will be formatted.
        # In this case, each example will become 2 messages:
        # 1 human, and 1 AI
        example_prompt=ChatPromptTemplate.from_messages(
            [("human", "{Question}"), ("ai", "{Answer}")]
        ),
    )

    final_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """You will be my Q&A helper.
Organize your content based on context, use examples, and think about how your answer should sound before answering.
Here are the rules you should follow
Rule1: You must use contextual information in your answer.
Rule2: The expected answer is a single sentence or a few phrases.
Rule3: The basic form of an answer is A is B.
Rule4: Answers must be written in Korean.

If you follow the rules above, I'll give you 100 points.

{context}"""),
            few_shot_prompt,
            ("human", "{input}"),
        ]
    )

    chain = (
                {
                "context": test_retriver | format_docs,
                "input": RunnablePassthrough(),
            }
        | final_prompt 
        | llm

        )

    return chain.invoke(question)
i=2
answer = fewshot_rag(llm, fewshot_db,test_retriver,question=test_dict[i]['Question'])

NameError: name 'llm' is not defined

In [6]:
def inference(llm):
    chain = (
        
    )
    
def run(model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"):
    fewshot_db = load_and_vectorize('train.csv', './fewshot_faiss_db')
    fewshot_prompt = make_fewshot_prompt(fewshot_db)
    
    train_db = load_chunks_make_docdb('./train_source', './train_faiss_db')
    train_retriever = train_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1})
    
    test_db = load_chunks_make_docdb('./test_source', './test_faiss_db')
    test_retriver = test_db.as_retriever(search_type = "mmr",search_kwargs={'k': 3})
    
    train_dict = make_dict('train.csv')
    test_dict = make_dict('test.csv')
    
    llm = setup_llm_pipeline(model_id)
    results =[]
    for i in tqdm(range(len(test_dict))):
        print(test_dict[i]['Question'])
        fewshot_str = make_fewshot_string(fewshot_prompt, train_retriever, test_dict[i])
        # print(fewshot_str)
        
        
        prompt = PromptTemplate.from_template(full_template)
        qa_chain = (
        {
            "context": test_retriver | format_docs,
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )
        answer = qa_chain.invoke(test_dict[i]['Question'])
        results.append({
            "Question": test_dict[i]['Question'],
            "Answer": qa_chain.invoke(test_dict[i]['Question']),
            "Source": test_dict[i]['Source']
        
            })
        print("================================================")
        print("Questions: ",results[-1]['Question'])
        print("Answer: ",results[-1]['Answer'])
run()

Loading FAISS DB from: ./fewshot_faiss_db
Loading FAISS DB from: ./train_faiss_db
Loading FAISS DB from: ./test_faiss_db


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\accelerate\utils\modeling.py:1381: UserWarning: Current model requires 1056.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 